In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os

import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision
import torchvision.transforms as transform

import albumentations as A
from albumentations.pytorch import ToTensorV2

from tqdm import tqdm

import matplotlib.pyplot as plt

%matplotlib inline

# 1. Load data

In [ ]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('/kaggle/input/ebhi-seg-csv/all/train_fold_2.csv')
train_data

In [ ]:
unique, frequency = np.unique(train_data.label, return_counts = True)
# print unique values array
print("Unique Values:", unique)

# print frequency array
print("Frequency Values:", frequency)

In [ ]:
import matplotlib.pyplot as plt

df_data = pd.DataFrame({'class': unique, 'count': frequency})
classes = df_data['class']
num_count = df_data['count']

fig = plt.figure(figsize=(7, 4))
plt.bar(classes, num_count, width=0.5)
plt.xticks(rotation = 30)

for index, value in enumerate(num_count):
  plt.text(index, value, str(value), rotation = 0, ha = 'center')

plt.title('Số lượng theo loại polyp', fontweight='bold')
plt.show()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_data.label = le.fit_transform(train_data.label)

# 2. Create Dataset, Dataloader

In [ ]:
#import numpy as np
from PIL import Image
#from torch.utils.data import Dataset, DataLoader

class PolypImageDataset(Dataset):
  """Image segmentation dataset."""

  def __init__(self, dataset, data_path, transform):
    """
    Args:
        dataset
    """
    self.dataset = dataset
    self.data_path = data_path
    self.transform = transform

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    image = Image.open(os.path.join(self.data_path, self.dataset.iloc[idx, 0]))
    label = self.dataset.iloc[idx, 2]

    if self.transform is not None:
        image = self.transform(image)

    return image, label

In [ ]:
def train_val_split(dataset, validation_split=0.2):
    """make dataloader for pytorch training

    Args:
        dataset ([object]): [the dataset object]
        train_batch_size ([int]): [training batch size]
        validation_split (float, optional): [validation ratio]. Defaults to 0.2.

    Returns:
        [type]: [description]
    """
    # number of samples in train and test set
    train_len = int(len(dataset) * (1 - validation_split))
    test_len = len(dataset) - train_len
    train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])
    # create train_loader
    #train_loader = torch.utils.data.DataLoader(train_set, batch_size=train_batch_size, shuffle=True,)
    # create test_loader
    #test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False,)
    return train_set, test_set

In [ ]:
def compute_mean_std(df_data, image_size, batch_size):
    transformer = transform.Compose([
       transform.Resize((image_size, image_size)),
       transform.ToTensor(),
       transform.Normalize(mean=(0,0,0), std=(1,1,1))
    ])
    
    # dataset
    image_dataset = PolypImageDataset(df_data, data_path='/kaggle/input/ebhi-seg/', transform=transformer)
    # data loader
    image_loader = DataLoader(image_dataset, batch_size=batch_size, shuffle= False, pin_memory  = True)
    
    # placeholders
    psum    = torch.tensor([0.0, 0.0, 0.0])
    psum_sq = torch.tensor([0.0, 0.0, 0.0])
    # loop through images
    for inputs,_ in image_loader:
        psum    += inputs.sum(axis = [0, 2, 3])
        psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])
    
    # pixel count
    count = len(df_data) * image_size * image_size

    # mean and std
    total_mean = psum / count
    total_var  = (psum_sq / count) - (total_mean ** 2)
    total_std  = torch.sqrt(total_var)
    
    return total_mean, total_std

In [ ]:
total_mean, total_std = compute_mean_std(train_data, image_size=224, batch_size=8)
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

In [ ]:
def dataloader(train_df, batch_size):
    train_data_0, val_data_0 = EmbryoData(train_df, current_fold=0).get_splits()
    train_data_1, val_data_1 = EmbryoData(train_df, current_fold=1).get_splits()
    train_data_2, val_data_2 = EmbryoData(train_df, current_fold=2).get_splits()
    train_data_3, val_data_3 = EmbryoData(train_df, current_fold=3).get_splits()
    train_data_4, val_data_4 = EmbryoData(train_df, current_fold=4).get_splits()

    train_dataloader_0 = DataLoader(train_data_0, batch_size=batch_size, shuffle = True)
    val_dataloader_0 = DataLoader(val_data_0, batch_size=batch_size, shuffle = False)
    
    train_dataloader_1 = DataLoader(train_data_1, batch_size=batch_size, shuffle = True)
    val_dataloader_1 = DataLoader(val_data_1, batch_size=batch_size, shuffle = False)
    
    train_dataloader_2 = DataLoader(train_data_2, batch_size=batch_size, shuffle = True)
    val_dataloader_2 = DataLoader(val_data_2,batch_size=batch_size, shuffle = False)
    
    train_dataloader_3 = DataLoader(train_data_3, batch_size=batch_size, shuffle = True)
    val_dataloader_3 = DataLoader(val_data_3, batch_size=batch_size, shuffle = False)
    
    train_dataloader_4 = DataLoader(train_data_4, batch_size=batch_size, shuffle = True)
    val_dataloader_4 = DataLoader(val_data_4, batch_size=batch_size, shuffle = False)
    
    dataloaders = [(train_dataloader_0, val_dataloader_0),
                   (train_dataloader_1, val_dataloader_1),
                   (train_dataloader_2, val_dataloader_2), 
                   (train_dataloader_3, val_dataloader_3),
                   (train_dataloader_4, val_dataloader_4)]
    
    return dataloaders

In [ ]:
import torchvision.transforms as transform
import torchvision

#mean = (0.5192, 0.3475, 0.6820)
#std = (0.2461, 0.2508, 0.1840)
IMG_SIZE = (224, 224)

transformer = transform.Compose([
    transform.Resize(IMG_SIZE),
    #transform.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transform.RandomHorizontalFlip(p=0.5),  
    transform.RandomVerticalFlip(p=0.5),
    #transform.RandomAffine(degrees=11, translate=(0.1,0.1), scale=(0.8, 0.8)),
    transform.ToTensor(),
    transform.Normalize(total_mean, total_std)
])

image_dataset = PolypImageDataset(train_data, data_path='/kaggle/input/ebhi-seg/', transform=transformer)
train_dataset, valid_dataset = train_val_split(image_dataset, validation_split=0.5)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle = False)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=8, shuffle = False)

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize = (12, 12))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.axis("off")
    plt.pause(0.001)

In [ ]:
# Visualization
images, classes = next(iter(train_loader))
# Make a grid from batch
out = torchvision.utils.make_grid(images, nrow=8)
imshow(out, title=classes)

# Define Metrics and Optimizers and Loss function

In [ ]:
from sklearn import metrics as skmetrics
import numpy

class Metrics:
    def __init__(self, metric_names):
        self.metric_names = metric_names
        # initialize a metric dictionary
        self.metric_dict = {metric_name: [0] for metric_name in self.metric_names}

    def step(self, labels, preds):
        for metric in self.metric_names:
            # get the metric function
            do_metric = getattr(
                skmetrics, metric, "The metric {} is not implemented".format(metric)
            )
            # check if metric require average method, if yes set to 'micro' or 'macro' or 'None'
            try:
                self.metric_dict[metric].append(
                    do_metric(labels, preds, average="macro")
                )
            except:
                self.metric_dict[metric].append(do_metric(labels, preds))

    def epoch(self):
        # calculate metrics for an entire epoch
        avg = [sum(metric) / (len(metric) - 1) for metric in self.metric_dict.values()]
        metric_as_dict = dict(zip(self.metric_names, avg))
        return metric_as_dict

    def last_step_metrics(self):
        # return metrics of last steps
        values = [self.metric_dict[metric][-1] for metric in self.metric_names]
        metric_as_dict = dict(zip(self.metric_names, values))
        return metric_as_dict

In [ ]:
train_metrics = Metrics(["accuracy_score", "f1_score"])
val_metrics = Metrics(["accuracy_score", "f1_score"])

# Define the Model: Transfer Learning

In [ ]:
from torchvision import models
from torch import nn
model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = True

classifier = nn.Sequential(
    nn.Linear(in_features=model.fc.in_features, out_features=250),
    nn.Linear(250, 100),
    nn.Linear(in_features=100, out_features=6)
)
model.fc  = classifier
print(classifier)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class LabelSmoothingLoss(nn.Module):
    def __init__(self, weight, epsilon=0.1, num_classes=2):
        super(LabelSmoothingLoss, self).__init__()
        self.epsilon = epsilon
        self.num_classes = num_classes
        self.weight = weight
        
    def forward(self, pred, target):
        smooth = pred.float()
        smooth = (1 - self.epsilon) * pred + self.epsilon / self.num_classes
        weights = self.weight.to(pred.device)
        return nn.CrossEntropyLoss(weight=weights)(smooth, target)

In [ ]:
# Đếm số lượng mẫu của mỗi lớp
num_classes = 6 
class_counts = torch.zeros(num_classes)

for _, label in train_loader:
    class_counts += torch.bincount(label, minlength=num_classes)
    
# Tính tổng số lượng mẫu
total_samples = len(train_loader.dataset)

# Tính class weights dựa trên số lượng mẫu của từng lớp
weights = total_samples / (num_classes * class_counts.float())
print("Class Weights:", weights)

In [ ]:
#criterion = LabelSmoothingLoss(weight=weights, epsilon=0.12, num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=2, factor=0.5)

# Define a training epoch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

In [ ]:
def train_one_epoch(
    model,
    train_loader,
    test_loader,
    device,
    optimizer,
    criterion,
    train_metrics,
    val_metrics,
):

    # training-the-model
    train_loss = 0
    valid_loss = 0
    all_labels = []
    all_preds = []
    model.train()
    for data, target in train_loader:
        # move-tensors-to-GPU
        data = data.type(torch.FloatTensor).to(device)
        # target=torch.Tensor(target)
        target = target.float().to(device)
        # clear-the-gradients-of-all-optimized-variables
        optimizer.zero_grad()
        # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
        output = model(data)
        # get the prediction label and target label
        #output = model(data)
        preds = torch.argmax(output, axis=1).cpu().detach().numpy()
        labels = target.cpu().numpy()
        # calculate-the-batch-loss
        loss = criterion(output.type(torch.FloatTensor), target.type(torch.LongTensor))
        # backward-pass: compute-gradient-of-the-loss-wrt-model-parameters
        loss.backward()
        # perform-a-ingle-optimization-step (parameter-update)
        optimizer.step()
        # update-training-loss
        train_loss += loss.item() * data.size(0)
        # calculate training metrics
        all_labels.extend(labels)
        all_preds.extend(preds)
    
    train_metrics.step(all_labels, all_preds)

    # validate-the-model
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for data, target in test_loader:
            data = data.type(torch.FloatTensor).to(device)
            target = target.to(device)
            output = model(data)
            preds = torch.argmax(output, axis=1).tolist()
            labels = target.tolist()
            all_labels.extend(labels)
            all_preds.extend(preds)
            loss = criterion(output, target)

            # update-average-validation-loss
            valid_loss += loss.item() * data.size(0)

    val_metrics.step(all_labels, all_preds)
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(test_loader.sampler)

    return (
        train_loss,
        valid_loss,
        train_metrics.last_step_metrics(),
        val_metrics.last_step_metrics(),
    )

# Training

In [ ]:
from tqdm import tqdm

num_epoch = 100
best_val_acc = 0

import logging
import numpy as np
print("begin training process")

for i in tqdm(range(0, num_epoch)):
    loss, val_loss, train_result, val_result = train_one_epoch(
        model,
        train_loader,
        valid_loader,
        device,
        optimizer,
        criterion,
        train_metrics,
        val_metrics,
    )

    scheduler.step(val_loss)
    #writer.add_scalar('Train/Loss', loss, i)
    #writer.add_scalar('Val/Loss', val_loss, i)
    #writer.add_scalar('Train/Acc', train_result, i)
    #writer.add_scalar('Val/Acc', val_result, i)
    print(
        "Epoch {} / {} \n Training loss: {} - Other training metrics: ".format(
            i + 1, num_epoch, loss
        )
    )
    print(train_result)
    print(
        " \n Validation loss : {} - Other validation metrics:".format(val_loss)
    )
    print(val_result)
    print("\n")
    # saving epoch with best validation accuracy
    if best_val_acc < float(val_result["accuracy_score"]):
        print(
            "Validation accuracy= "+
            str(val_result["accuracy_score"])+
            "===> Save best epoch"
        )
        best_val_acc = val_result["accuracy_score"]
        torch.save(model.state_dict(),"./" +  "best_resnet_2.pt",)
    else:
        print("Validation accuracy= "+ str(val_result["accuracy_score"])+ "===> No saving")
        continue

# Test the results

In [ ]:
test_data = pd.read_csv('/kaggle/input/ebhi-seg-csv/all/test_fold_2.csv')
test_data

In [ ]:
from sklearn import preprocessing
test_data.label = le.transform(test_data.label)

In [ ]:
total_mean, total_std = compute_mean_std(test_data, image_size=224, batch_size=8)
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

In [ ]:
test_transform = transform.Compose([
    transform.Resize((224, 224)),
    transform.ToTensor(),
    transform.Normalize(total_mean, total_std)
])

test_dataset = PolypImageDataset(test_data, data_path='/kaggle/input/ebhi-seg', transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
import copy
test_model = copy.deepcopy(model)
test_model.load_state_dict(torch.load("best_resnet_2.pt"))
test_model = test_model.to(device)

In [ ]:
def test_result(model, test_loader, device):
    # testing the model by turning model "Eval" mode
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for data, target in test_loader:
            # move-tensors-to-GPU
            data = data.to(device)
            target = target.to(device)
            # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
            output = model(data)
            # applying Softmax to results
            prob = nn.Softmax(dim=1)
            probs = prob(output)
            labels.extend(target.tolist())
            preds.extend(torch.argmax(probs, axis=1).tolist())
    return preds, labels

In [ ]:
preds, grd_truth = test_result(test_model, test_loader, device)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
report = classification_report(grd_truth, preds, digits=6,target_names=le.classes_)
print(report)

In [ ]:
cm = confusion_matrix(grd_truth, preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot()